In [5]:
# ========== WPV Data Cleaning Script ==========

import pandas as pd
import numpy as np

# ========== 1. Load Data ==========
df_phase1 = pd.read_excel("../../prepare/WVP Data Collection - Phase I - M.xlsm", sheet_name=0, skiprows=1)
df_december = pd.read_excel("../../prepare/MH December 2024 Data.xlsx", sheet_name="MHA WPV Data Export December 20", header=4)
df_aug_oct = pd.read_excel("../../prepare/MHA WPV events August through October 2024.xlsx", sheet_name="MHA WPV events August through O", header=1)


df_phase1['source'] = 'phase1'
df_december['source'] = 'december'
df_aug_oct['source'] = 'aug_oct'

# ========== 2. Remove Unnamed Columns ==========
df_phase1 = df_phase1.loc[:, ~df_phase1.columns.str.contains('^Unnamed')]
df_december = df_december.loc[:, ~df_december.columns.str.contains('^Unnamed')]
df_aug_oct = df_aug_oct.loc[:, ~df_aug_oct.columns.str.contains('^Unnamed')]

# ========== 3. Rename Columns ==========
map_common = {
    'Event Date': 'event_time',
    'Occupational Category of Person Affected': 'victim_profession',
    'Department/Office Incident Took Place': 'department',
    'Aggressor': 'perpetrator_type',
    'Type of Violence': 'violence_type',
    'Severity of Assault': 'severity',
    'Emotional and/ or Psychological Impact': 'emotional_impact',
    'Level of Care Needed': 'physical_injury_level',
    'Primary Assault Description': 'assault_desc',
    'Assault Description1': 'assault_detail',
    'Response Action Taken': 'response_action',
    'Primary Contributing Factors': 'contributing_factors',
    'Incident #': 'incident_id'
}

map_aug_oct = {
    'victim_profession': 'victim_profession',
    'victim_primary_job': 'victim_primary_job',
    'event_time': 'event_time',
    'violence_type': 'violence_type',
    'contributing_factors': 'contributing_factors',
    'emotional_impact': 'emotional_impact',
    'severity': 'severity',
    'severity_level': 'severity_level',
    'department': 'department',
    'response_action': 'response_action',
    'perpetrator_type': 'perpetrator_type',
    'assault_desc': 'assault_desc',
    'osha_recordable': 'osha_recordable',
    'days_missed': 'days_missed',
}

df_phase1 = df_phase1.rename(columns=map_common)
df_december = df_december.rename(columns=map_common)
df_aug_oct = df_aug_oct.rename(columns=map_aug_oct)

if 'severity_level' not in df_aug_oct.columns and 'severity' in df_aug_oct.columns:
    df_aug_oct['severity_level'] = df_aug_oct['severity'].apply(standardize_severity)

# ========== 4. Extract victim_primary_job ==========
for df in [df_phase1, df_december]:
    if 'victim_profession' in df.columns:
        df['victim_primary_job'] = df['victim_profession'].astype(str).str.split(',').str[0].str.strip()

# ========== 5. Standardize Department ==========
def standardize_department(dep):
    if pd.isna(dep):
        return "Unknown"
    dep = str(dep).lower()
    if any(x in dep for x in ['ed', 'er', 'hallway']):
        return "Emergency Department"
    elif 'icu' in dep:
        return "ICU"
    elif 'ach' in dep:
        return "ACH"
    else:
        return dep.strip().title()

for df in [df_phase1, df_december, df_aug_oct]:
    if 'department' in df.columns:
        df['department'] = df['department'].apply(standardize_department)

# Define severity standardization
def standardize_severity(value):
    if pd.isna(value):
        return 'Unknown'
    value = str(value).lower()
    if 'none' in value:
        return 'None'
    elif 'mild' in value:
        return 'Mild'
    elif 'moderate' in value:
        return 'Moderate'
    elif 'severe' in value:
        return 'Severe'
    else:
        return 'Unknown'

# Apply to relevant columns
df_phase1['severity_level'] = df_phase1['severity'].apply(standardize_severity)
df_december['severity_level'] = df_december['severity'].apply(standardize_severity)


# ========== 6. Align Columns ==========
columns_to_keep = [
    'event_time', 'victim_profession', 'victim_primary_job',
    'department', 'perpetrator_type', 'violence_type',
    'severity', 'emotional_impact', 'physical_injury_level',
    'assault_desc', 'response_action', 'contributing_factors',
    'osha_recordable', 'days_missed', 'source'
]

def align_columns(df, columns):
    for col in columns:
        if col not in df.columns:
            df[col] = pd.NA
    return df[columns]

df_phase1 = align_columns(df_phase1, columns_to_keep)
df_december = align_columns(df_december, columns_to_keep)
df_aug_oct = align_columns(df_aug_oct, columns_to_keep)


# ===== Fill Missing Values for Key Columns =====

# Fill missing values safely using .loc
for df in [df_phase1, df_december, df_aug_oct]:
    if 'severity' in df.columns:
        df.loc[:, 'severity'] = df['severity'].fillna("Unknown")
    if 'department' in df.columns:
        df.loc[:, 'department'] = df['department'].fillna("Unknown")


# ========== 7. Merge ==========
df_all = pd.concat([df_phase1, df_december, df_aug_oct], ignore_index=True)

# ========== 8. Save ==========
df_all.to_csv("../../cleaned_data/lily/merged_wpv_cleaned.csv", index=False)
print("✅ Cleaned & merged WPV dataset saved to merged_wpv_cleaned.csv")


✅ Cleaned & merged WPV dataset saved to merged_wpv_cleaned.csv
